## Victor Manuel Gómez Espinosa
### victor.gomez@cimat.mx


# Dirección de los archivos

In [1]:
#librerias
import os
CurrentPath = '/content/drive/My Drive/KUESKI_Challenge'

os.chdir(CurrentPath) #cambiar la ubicacion a la carpeta donde se encuentra este archivo
!ls #listar los documentos en esta ruta

1_Basic_exploratory_analysis.ipynb	listings_reduced_completeFeatures.csv
2_Model_implementation.ipynb		listings_reduced.csv
datos.csv				_missing_value.py
EliminandoCasosIncomp_Challenge2.ipynb	__pycache__
Imputando_Challenge.ipynb		Resultados


# Librerías

In [2]:
# librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from datetime import datetime
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_val_score
import joblib
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import median_absolute_error

# Datos

In [3]:
df=pd.read_csv('datos.csv',index_col=0,na_values=['unk','unknown',' ','  ',None]) #leemos los datos

In [7]:
#sólo eliminar esa observación atipica de precio 0
df=df.drop(20466057)

In [8]:
X, y= df.drop(columns=['price']).values, df.price.values #definimos las variables

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42) #dividimos el conjunto de prueba y entrenamiento

# Implementación del modelo

Observaciones:

- Del paso anterior, como se mencionó parece que hay datos atípicos y como no sabemos si son de particular interés para el  negocio, podemos utilizar la métrica MAE (mean absolute error) para que no se vea tan afectado por estos.
-Como hay valores faltantes, y XGBoost puede manejarlos sin necesidad de imputación, además de que se pueden aprovechar las ventajas de ser paralelizable y aprovechar la GPU para hacer pruebas rápidas, proporcionar buenos resultados y obtener las características más importantes en el modelo.

Probamos con la configuración inicial de XGBoost

In [ ]:
startTime = datetime.now()
xgbCW=xgb.XGBRegressor(seed=123,sampling_method='gradient_based',tree_method='gpu_hist', predictor='gpu_predictor',learning_rate=0.3,n_estimators=500,subsample=0.9,gamma=0,reg_lambda=1,colsample_bytree=1,colsample_bylevel=1,colsample_bynode=1,max_depth=6,min_child_weight=1,n_jobs=-1)
                    
hist=xgbCW.fit(X_train,y_train,eval_set=[(X_test,y_test)],eval_metric='mae',early_stopping_rounds=50)

print(datetime.now() - startTime)

[23:22:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:45.4124
Will train until validation_0-mae hasn't improved in 50 rounds.
[1]	validation_0-mae:33.353
[2]	validation_0-mae:27.2172
[3]	validation_0-mae:24.3094
[4]	validation_0-mae:23.2128
[5]	validation_0-mae:22.8187
[6]	validation_0-mae:22.5737
[7]	validation_0-mae:22.9229
[8]	validation_0-mae:22.7271
[9]	validation_0-mae:22.66
[10]	validation_0-mae:22.6725
[11]	validation_0-mae:22.4356
[12]	validation_0-mae:22.5689
[13]	validation_0-mae:22.7268
[14]	validation_0-mae:22.683
[15]	validation_0-mae:22.8799
[16]	validation_0-mae:23.0835
[17]	validation_0-mae:23.0544
[18]	validation_0-mae:22.8613
[19]	validation_0-mae:22.9236
[20]	validation_0-mae:22.959
[21]	validation_0-mae:22.8578
[22]	validation_0-mae:22.8475
[23]	validation_0-mae:22.7915
[24]	validation_0-mae:22.748
[25]	validation_0-mae:22.7667
[26]	validation_0-mae:22.953
[27]	validation_

Evaluación

In [ ]:
# predecir datos de prueba
y_pred_train_xgb =xgbCW.predict(X_train)
y_pred_test_xgb = xgbCW.predict(X_test)

In [ ]:
#train
print('train')
mse=mean_squared_error(y_train,y_pred_train_xgb)
r2=r2_score(y_train,y_pred_train_xgb)
mae=median_absolute_error(y_train,y_pred_train_xgb)
print('RMSE: ',(mse)**(0.5),' r2: ',r2,' mae: ',mae)

train
RMSE:  48.24012056014505  r2:  0.9068311987777398  mae:  12.047382354736328


In [ ]:
#test
print('test')
mse=mean_squared_error(y_test,y_pred_test_xgb)
r2=r2_score(y_test,y_pred_test_xgb)
mae=median_absolute_error(y_test,y_pred_test_xgb)
print('RMSE: ',(mse)**(0.5),' r2: ',r2,' mae: ',mae)

test
RMSE:  48.32533380181864  r2:  0.9047391712329087  mae:  12.494937896728516


# Ajustamos hiper parametros

Ajustamos hiper parámetros, mediante búsqueda en malla con 10-folds:
- primero ajustamos learning rate y un número aproximado de estimadores
- después profundidad máxima del árbol
- después submuestreo para construir el árbol
- por último, regularización  


In [ ]:
#xgb
startTime = datetime.now()
def XGBoost_model():
    
    gbm_param_grid={'n_estimators':[5,10,25],
                    'learning_rate':[1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,0.3,0.5,0.9], #1e-5,1e-4,1e-3,1e-2,1e-1,0.3,0.5,0.9
               'max_depth':[6],
               'min_child_weight':[1],
               'scale_pos_weight' : [1],

               'subsample':[1],
                'colsample_bytree':[1],
                'colsample_bylevel':[1],
                'colsample_bynode':[1],

               'gamma':[0],
               'lambda':[1]
               
               }
    
    
    gbm=xgb.XGBRegressor(seed=123,tree_method='gpu_hist', predictor='gpu_predictor')
                

    
    return gbm, gbm_param_grid 

Regressor,params=XGBoost_model()
model = GridSearchCV(Regressor,param_grid=params,scoring='neg_mean_absolute_error',cv=10,n_jobs=-1)
model.fit(X_train,y_train)

# Print the tuned parameters and score
print("Tuned XGB Parameters: {}".format(model.best_params_)) 
print("Best score is {}".format(model.best_score_))
print(datetime.now() - startTime)

[21:55:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tuned XGB Parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 25, 'scale_pos_weight': 1, 'subsample': 1}
Best score is -25.392881656059167
0:00:41.858082


In [ ]:
#xgb
startTime = datetime.now()
def XGBoost_model():
    
    gbm_param_grid={'n_estimators':[25],
                    'learning_rate':[0.1], #1e-5,1e-4,1e-3,1e-2,1e-1,0.3,0.5,0.9
               'max_depth':[7,8,9,10,11,12,13,14],
               'min_child_weight':[1],
               'scale_pos_weight' : [1],

               'subsample':[1],
                'colsample_bytree':[1],
                'colsample_bylevel':[1],
                'colsample_bynode':[1],

               'gamma':[0],
               'lambda':[1]
               
               }
    
    
    gbm=xgb.XGBRegressor(seed=123,tree_method='gpu_hist', predictor='gpu_predictor')
                

    
    return gbm, gbm_param_grid 

Regressor,params=XGBoost_model()
model = GridSearchCV(Regressor,param_grid=params,scoring='neg_mean_absolute_error',cv=10,n_jobs=-1)
model.fit(X_train,y_train)

# Print the tuned parameters and score
print("Tuned XGB Parameters: {}".format(model.best_params_)) 
print("Best score is {}".format(model.best_score_))
print(datetime.now() - startTime)

[22:02:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tuned XGB Parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 1, 'n_estimators': 25, 'scale_pos_weight': 1, 'subsample': 1}
Best score is -23.24788804718798
0:01:14.287792


In [ ]:
#xgb
startTime = datetime.now()
def XGBoost_model():
    
    gbm_param_grid={'n_estimators':[25],
                    'learning_rate':[0.1], #1e-5,1e-4,1e-3,1e-2,1e-1,0.3,0.5,0.9
               'max_depth':[12],
               'min_child_weight':[1],
               'scale_pos_weight' : [1],

               'subsample':[0.8,0.9,1],
                'colsample_bytree':[0.8,0.9,1],
                'colsample_bylevel':[1],
                'colsample_bynode':[1],

               'gamma':[0],
               'lambda':[1]
               
               }
    
    
    gbm=xgb.XGBRegressor(seed=123,tree_method='gpu_hist', predictor='gpu_predictor')
                

    
    return gbm, gbm_param_grid 

Regressor,params=XGBoost_model()
model = GridSearchCV(Regressor,param_grid=params,scoring='neg_mean_absolute_error',cv=10,n_jobs=-1)
model.fit(X_train,y_train)

# Print the tuned parameters and score
print("Tuned XGB Parameters: {}".format(model.best_params_)) 
print("Best score is {}".format(model.best_score_))
print(datetime.now() - startTime)

[23:07:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tuned XGB Parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'lambda': 1, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 1, 'n_estimators': 25, 'scale_pos_weight': 1, 'subsample': 1}
Best score is -23.24788804718798
0:01:53.110610


In [ ]:
#xgb
startTime = datetime.now()
def XGBoost_model():
    
    gbm_param_grid={'n_estimators':[25],
                    'learning_rate':[0.1], #1e-5,1e-4,1e-3,1e-2,1e-1,0.3,0.5,0.9
               'max_depth':[12],
               'min_child_weight':[1],
               'scale_pos_weight' : [1],

               'subsample':[1],
                'colsample_bytree':[1],
                'colsample_bylevel':[1],
                'colsample_bynode':[1],

               'gamma':[0,0.1,1,10,100],
               'lambda':[0,0.1,1,10,100]
               
               }
    
    
    gbm=xgb.XGBRegressor(seed=123,tree_method='gpu_hist', predictor='gpu_predictor')
                

    
    return gbm, gbm_param_grid 

Regressor,params=XGBoost_model()
model = GridSearchCV(Regressor,param_grid=params,scoring='neg_mean_absolute_error',cv=10,n_jobs=-1)
model.fit(X_train,y_train)

# Print the tuned parameters and score
print("Tuned XGB Parameters: {}".format(model.best_params_)) 
print("Best score is {}".format(model.best_score_))
print(datetime.now() - startTime)

[22:58:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Tuned XGB Parameters: {'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'lambda': 0, 'learning_rate': 0.1, 'max_depth': 12, 'min_child_weight': 1, 'n_estimators': 25, 'scale_pos_weight': 1, 'subsample': 1}
Best score is -23.24788804718798
0:04:45.113551


In [ ]:
# predecir datos de prueba
y_pred_train_xgb =model.predict(X_train)
y_pred_test_xgb = model.predict(X_test)

In [ ]:
#test
print('test')
mse=mean_squared_error(y_test,y_pred_test_xgb)
r2=r2_score(y_test,y_pred_test_xgb)
mae=median_absolute_error(y_test,y_pred_test_xgb)
print('RMSE: ',(mse)**(0.5),' r2: ',r2,' mae: ',mae)

test
RMSE:  49.11123077220083  r2:  0.9016155942912647  mae:  10.114242553710938


Mismos parámetros con early stopping

In [ ]:
startTime = datetime.now()
xgbCW=xgb.XGBRegressor(seed=123,sampling_method='gradient_based',tree_method='gpu_hist', predictor='gpu_predictor',learning_rate=0.1,n_estimators=500,subsample=1,gamma=0,reg_lambda=1,colsample_bytree=1,colsample_bylevel=1,colsample_bynode=1,max_depth=12,min_child_weight=1,n_jobs=-1)
                    
hist=xgbCW.fit(X_train,y_train,eval_set=[(X_test,y_test)],eval_metric='mae',early_stopping_rounds=50)

print(datetime.now() - startTime)

[23:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:57.8105
Will train until validation_0-mae hasn't improved in 50 rounds.
[1]	validation_0-mae:52.2638
[2]	validation_0-mae:47.3158
[3]	validation_0-mae:43.0327
[4]	validation_0-mae:39.3235
[5]	validation_0-mae:35.9872
[6]	validation_0-mae:33.1575
[7]	validation_0-mae:30.8431
[8]	validation_0-mae:28.8988
[9]	validation_0-mae:27.174
[10]	validation_0-mae:25.7209
[11]	validation_0-mae:24.555
[12]	validation_0-mae:23.6895
[13]	validation_0-mae:23.0117
[14]	validation_0-mae:22.5298
[15]	validation_0-mae:22.1203
[16]	validation_0-mae:21.7955
[17]	validation_0-mae:21.4798
[18]	validation_0-mae:21.3072
[19]	validation_0-mae:21.1475
[20]	validation_0-mae:21.0053
[21]	validation_0-mae:20.9111
[22]	validation_0-mae:20.8782
[23]	validation_0-mae:20.7941
[24]	validation_0-mae:20.7092
[25]	validation_0-mae:20.6771
[26]	validation_0-mae:20.6329
[27]	valida

In [ ]:
# predecir datos de prueba
y_pred_train_xgb =xgbCW.predict(X_train)
y_pred_test_xgb = xgbCW.predict(X_test)
y_pred = xgbCW.predict(X)

In [ ]:
#train
print('train')
mse=mean_squared_error(y_train,y_pred_train_xgb)
r2=r2_score(y_train,y_pred_train_xgb)
mae=median_absolute_error(y_train,y_pred_train_xgb)
print('RMSE: ',(mse)**(0.5),' r2: ',r2,' mae: ',mae)

train
RMSE:  35.11256837729525  r2:  0.9506395699910057  mae:  7.746747970581055


In [ ]:
#test
print('test')
mse=mean_squared_error(y_test,y_pred_test_xgb)
r2=r2_score(y_test,y_pred_test_xgb)
mae=median_absolute_error(y_test,y_pred_test_xgb)
print('RMSE: ',(mse)**(0.5),' r2: ',r2,' mae: ',mae)

test
RMSE:  47.87686317300886  r2:  0.906499053544028  mae:  10.062044143676758


Caracteristicas más importantes

In [ ]:
feat_names=np.array(list(df4.drop(columns=['price']).columns))
importances=xgbCW.feature_importances_

d = {"Caracteristicas": feat_names,
      "Importances(Gain)": importances
      }

dfimp=pd.DataFrame(d).sort_values(by=['Importances(Gain)']).tail(10)

px.bar(dfimp, x="Importances(Gain)", y="Caracteristicas", title="10 Características más importantes",orientation='h')